In [1]:
import casadi as ca
import numpy as np
import ciropt as co
import sympy as sp

In [2]:
L_smooth = 1.
mu = 0.0001
Capacitance = 2.
Inductance = 2.
R = 1.

solver = "ca_canonical"
# solver = "ca_canonical_X"

In [3]:
problem = co.accelerated_gradient_circuit(mu, L_smooth, R, Capacitance, Inductance)
problem.obj = problem.b + problem.d
res, sol, sp_exp = problem.solve(solver=solver, verbose=False, debug=True)[:3]

dim_G=6, dim_F=4


In [4]:
res

{'b': 2.66846146628235,
 'h': 6.035347263881326,
 'd': 0.7721222423737911,
 'alpha': 0.4915885826616601,
 'beta': 0.4437526051891429}

In [5]:
core_vars = sorted(['alpha', 'beta', 'h', 'b', 'd'])

if 'ca_vars' in globals():
    v = []
    core_vars_set = set(core_vars)
    for name in sorted(ca_vars.keys()):
        if "lamb" not in name and "P" not in name:
            v += [name]
            assert set(name.split("_")).issubset(core_vars_set)
    print("PASSED")
    v = sorted(v)
    sp_v = np.array([1] + [sp.symbols(name) for name in v])
    name2idx = {var.name:idx+1 for idx, var in enumerate(sp_v[1:])}
    name2idx[1] = 0
else:
    sp_v = np.array([1] + [sp.symbols(name) for name in problem.v_names[1:]])
    name2idx = problem.name2idx

In [6]:
sp_exp.keys()

dict_keys([(0, 0, 1), (0, 0, 2), (0, 0, 3), (0, 1, 0), (0, 1, 2), (0, 1, 3), (0, 2, 0), (0, 2, 1), (0, 2, 3), (0, 3, 0), (0, 3, 1), (0, 3, 2), 'FG_d'])

In [7]:
for k in sp_exp.keys():
    for mat_expr in sp_exp[k].values():
        coeff_matrix = co.linear_matrix_expr_to_coeff_matrix(mat_expr, name2idx)
        assert co.equal_sp_arrays(co.coeff_matrix_to_linear_matrix_expr(coeff_matrix, \
                                                                sp_v, mat_expr.shape), mat_expr)
    
print("PASSED")

PASSED


In [8]:
# check coefficient matrices, ie, A @ v == \nabla l(F, G)
v_coeffs, v_names, name2idx2, v_k_list = co.sp_v_coeff_matrix(sp_exp, core_vars)
sp_v2 = np.array([1] + [sp.symbols(name) for name in v_names[1:]])

for i, k in enumerate(v_k_list):
    for mtype, mat_expr in sp_exp[k].items():
        coeff_matrix = v_coeffs[mtype][i]
        assert co.equal_sp_arrays(co.coeff_matrix_to_linear_matrix_expr(coeff_matrix, sp_v2, mat_expr.shape), mat_expr)
    
print("PASSED")

PASSED


In [9]:
def get_start_idx(k):
    if k == 0:
        return 0
    return k * (k-1) // 2 + 1
a = np.zeros((10, 10))
count = 0
for i in range(10):
    a[i, :i+1] = np.arange(count, count + i + 1)
    count += i + 1
print(a)
vec_diag_P_idx = np.cumsum(np.arange(10 + 1))[1:] - 1
Q = np.zeros((10, 10 * (10 + 1) // 2))
np.put_along_axis(Q, vec_diag_P_idx.reshape(-1, 1), 1, axis=1)
vec_a = np.arange(10 * (10 + 1) // 2)
diag_a = Q @ vec_a

assert np.allclose(diag_a, np.diag(a))

for k1 in range(10):
    for k2 in range(10):
        assert np.allclose( (a @ a.T)[k1, k2], co.get_PPt_element(vec_a, k1, k2))

print("PASSED")


[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  2.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 3.  4.  5.  0.  0.  0.  0.  0.  0.  0.]
 [ 6.  7.  8.  9.  0.  0.  0.  0.  0.  0.]
 [10. 11. 12. 13. 14.  0.  0.  0.  0.  0.]
 [15. 16. 17. 18. 19. 20.  0.  0.  0.  0.]
 [21. 22. 23. 24. 25. 26. 27.  0.  0.  0.]
 [28. 29. 30. 31. 32. 33. 34. 35.  0.  0.]
 [36. 37. 38. 39. 40. 41. 42. 43. 44.  0.]
 [45. 46. 47. 48. 49. 50. 51. 52. 53. 54.]]
PASSED


In [10]:
for _ in range(10):
    dim_G = 6
    x = np.random.randn(100, 1)
    vec_indices = { "v"   : [0, 9],\
                            "lamb": [10, 70], \
                            "P"  : [71, 71 + dim_G * (dim_G + 1)//2]} 
    var_name = "P"
    l = co.get_vec_var(x, var_name, vec_indices, matrix=False)
    S = co.get_vec_var(x, var_name, vec_indices, matrix=True)
    vec_P = co.get_vec_var(x, "P", vec_indices)
    for k1 in range(dim_G):
        for k2 in range(dim_G):
            val = co.get_PPt_element(vec_P, k1, k2)
            S1, S2 = co.get_PPt_matrix(x, vec_indices, k1, k2)
            assert np.allclose(val, x.T @ S1.T @ S2 @ x)
    assert np.allclose(S @ x, l)
print("PASSED")

PASSED
